In [1]:
# sport_classification_no_transfer.ipynb

# ====================================
# 1. IMPORTY I USTAWIENIA
# ====================================
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

print(tf.__version__)  # Upewnij się, że TensorFlow działa poprawnie

# Ustawienia
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6  # 6 sportów
EPOCHS = 20      # Bez transfer learningu prawdopodobnie trzeba więcej epok
# (ale to zależy od liczby obrazów)

# ====================================
# 2. GENERATORY DANYCH
# ====================================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    directory='data/valid',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory='data/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ====================================
# 3. ZBUDOWANIE SIECI CNN OD ZERA
# ====================================
model = models.Sequential()

# Warstwa 1: Convolution + MaxPooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Warstwa 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Warstwa 3
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Spłaszczenie do wektora i warstwa Dense
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))  # Dropout pomaga w walce z overfittingiem
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.summary()

# ====================================
# 4. KOMPILACJA
# ====================================
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# ====================================
# 5. TRENING
# ====================================
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)

# ====================================
# 6. EWALUACJA
# ====================================
val_loss, val_accuracy = model.evaluate(valid_generator)
print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

# ====================================
# 7. ZAPISANIE MODELU
# ====================================
model.save("sport_classifier_no_transfer.keras")


2.18.0
Found 901 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


C:\Users\barte\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,734 (42.61 MB)

...

 Non-trainable params: 0 (0.00 B)

C:\Users\barte\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.2681 - loss: 2.2708

C:\Users\barte\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 366ms/step - accuracy: 0.2711 - loss: 2.2526 - val_accuracy: 0.5667 - val_loss: 1.2830
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 269ms/step - accuracy: 0.5448 - loss: 1.2317 - val_accuracy: 0.5667 - val_loss: 1.1840
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 268ms/step - accuracy: 0.5410 - loss: 1.1202 - val_accuracy: 0.5000 - val_loss: 1.2591
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 273ms/step - accuracy: 0.6395 - loss: 0.9185 - val_accuracy: 0.5000 - val_loss: 1.6396
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 266ms/step - accuracy: 0.6820 - loss: 0.8405 - val_accuracy: 0.7333 - val_loss: 0.7857
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 266ms/step - accuracy: 0.6397 - loss: 0.8675 - val_accuracy: 0.8333 - val_loss: 0.6698
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 266ms/step - accuracy: 0.6723 - loss: 0.8904 - val_accuracy: 0.7333 - val_loss: 0.8582
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 268ms/step - accuracy: 0.7488 - loss: 0.6559 - val_accuracy: 0.6000 - val

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Ścieżka do zapisanego modelu
MODEL_PATH = "sport_classifier_no_transfer.keras"

# Nazwy klas w takiej kolejności, w jakiej były trenowane w generatorze
class_names = [
    "baseball",
    "cricket",
    "mushing",
    "nascar racing",
    "rock climbing",
    "swimming"
]

# Wczytanie wytrenowanego modelu
model = load_model(MODEL_PATH)

def predict_sport(image_path, model, class_names):
    """
    Funkcja wczytuje pojedynczy obraz, przetwarza go
    i zwraca przewidywaną etykietę (oraz pewność).
    """
    # 1. Wczytanie obrazu i przeskalowanie do rozmiaru 224x224
    img = load_img(image_path, target_size=(224, 224))

    # 2. Konwersja do tablicy NumPy i normalizacja do zakresu [0, 1]
    img_array = img_to_array(img) / 255.0

    # 3. Dodanie wymiaru batch (1, 224, 224, 3), aby sieć mogła przetwarzać dane
    img_array = np.expand_dims(img_array, axis=0)

    # 4. Przewidywanie
    predictions = model.predict(img_array)  # shape: (1, NUM_CLASSES)

    # 5. Indeks klasy z najwyższym prawdopodobieństwem
    predicted_index = np.argmax(predictions[0])
    predicted_label = class_names[predicted_index]

    # Opcjonalnie pewność predykcji (confidence)
    confidence = predictions[0][predicted_index]

    return predicted_label, confidence

# Ścieżka do przykładowego zdjęcia
# (zmień na własną ścieżkę, np. 'test_images/my_image.jpg')
test_image_path = "image6.jpg"

# Klasyfikacja
label, conf = predict_sport(test_image_path, model, class_names)

print(f"Przewidywany sport: {label}")
print(f"Pewność modelu: {conf * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Przewidywany sport: nascar racing
Pewność modelu: 91.52%
